# Проект "сервис для чтения книг по подписке"

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задача:
1) [Посчитайте, сколько книг вышло после 1 января 2000 года;](#section_1)

2) [Для каждой книги посчитайте количество обзоров и среднюю оценку;](#section_2)

3) [Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;](#section_3)

4) [Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;](#section_4)

5) [Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.](#section_5)

Описание данных:

Таблица books содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

Таблица ratings содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Выполним подключение к базе данных.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Исследуем таблицы, напечатаем первые строки таблиц.

In [4]:
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Из выведенных данных видно, что они соответствуют описанию и обозначенной структуре.

## Посчитайте, сколько книг вышло после 1 января 2000 года.


In [9]:
query = '''SELECT COUNT(DISTINCT book_id)
FROM books
WHERE CAST(publication_date AS date) > '2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

<a id='section_2'><a>

## Для каждой книги посчитайте количество обзоров и среднюю оценку;


Для подсчета количества обзоров объединим таблицу books с таблицей reviews, для подсчета средней оценки - с таблицей ratings, объединим по колонке book_id.

In [10]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT r.review_id),
       ROUND(AVG(ra.rating), 1)
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS ra ON b.book_id = ra.book_id
GROUP BY b.book_id
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count,round
0,1,'Salem's Lot,2,3.7
1,2,1 000 Places to See Before You Die,1,2.5
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.7
3,4,1491: New Revelations of the Americas Before C...,2,4.5
4,5,1776,4,4.0
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.7
996,997,Xenocide (Ender's Saga #3),3,3.4
997,998,Year of Wonders,4,3.2
998,999,You Suck (A Love Story #2),2,4.5


Количество обзоров по книге в колонке count, средняя оценка в колонке round.

<a id='section_3'><a>

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;


In [11]:
query = '''
SELECT p.publisher,
       COUNT(b.book_id)
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY COUNT(DISTINCT b.book_id) DESC
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19
...,...,...
329,Random House Anchor,1
330,Random House Audio Publishing Group,1
331,Random House Large Print Publishing,1
332,Random House Trade,1


Наибольшее количество книг выпустило издательство Penguin Books - 42 книги.

<a id='section_4'><a>

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;


In [17]:
query = '''
WITH rating_50 AS (
SELECT book_id,
       COUNT(rating)
FROM ratings
GROUP BY book_id
HAVING COUNT(rating) >= 50
ORDER BY COUNT(rating) DESC
),

books_AVG_rating AS (
SELECT book_id,
       ROUND(AVG(rating),2) AS rating_avg
FROM ratings
GROUP BY book_id
ORDER BY ROUND(AVG(rating),2) DESC
), 
 
 books_rating AS (
SELECT r.book_id,
       rating_avg
FROM rating_50 AS r
LEFT JOIN books_AVG_rating AS b ON r.book_id = b.book_id
),

 books_author AS (
SELECT book_id,
       author
FROM books AS b
LEFT JOIN authors AS a ON b.author_id = a.author_id
)

SELECT author,
       ROUND(AVG(rating_avg),2)
FROM books_rating AS br
LEFT JOIN books_author AS ba ON br.book_id = ba.book_id
GROUP BY author
ORDER BY ROUND(AVG(rating_avg),2) DESC
LIMIT 1

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Автор с самой высокой средней оценкой книг оказался J.K. Rowling/Mary GrandPré, оценка 4.29.

<a id='section_5'><a>

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''
WITH rating_greater48 AS (
SELECT username,
       COUNT(rating_id)
FROM ratings
GROUP BY username
HAVING COUNT(rating_id) > 48
),

 user_reviews_count AS (
SELECT username,
       COUNT(review_id) AS reviews_count
FROM reviews 
GROUP BY username
),

reviews_count_by_users AS (
SELECT rg.username,
       reviews_count
FROM rating_greater48 AS rg
INNER JOIN user_reviews_count AS us ON rg.username = us.username
)

SELECT AVG(reviews_count)
FROM reviews_count_by_users

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24.

Вывод:
- После 1 января 2000 года вышло 818 книг.
- Найдено количество обзоров (в колонке count) и средняя оценка (в колонке round) по каждой книге.
- Наибольшее количество книг выпустило издательство Penguin Books - 42 книги.
- Автор с самой высокой средней оценкой книг оказался J.K. Rowling/Mary GrandPré, оценка 4.29.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24.

На основе имеющейся базы данных можно:
- найти популярные по отзывам книги и сделать по ним рекламное предложение,
- по авторам книг, имеющим высокий рейтинг, при выходе новой книги также можно сделать рекламу с акцентом на автора,
- маркетинговое предложение можно сделать основываясь на издательстве, которое выпустила наибольшее количество книг,
- также можно собрать книги с наибольшим количеством отзывов, иногда протеворечивых, чтобы вызвать интерес у читателей.